In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandasql
import gc

In [2]:
import missingno as msno

In [3]:
from pandasql import sqldf

In [4]:
from tqdm import tqdm

In [5]:
import sys

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# METADATA

In [7]:
def reduce_mem_usage(df):

    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    #create a dataframe and optimize its memory usage
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

# TRAIN AND TEST

In [8]:
train = pd.read_csv('training_set.csv',dtype={'object_id':np.int32,'mjd':np.float16,\
                                                   'passband':np.int8,'flux':np.float32,'flux_err':np.float32,'detected':np.int8})

In [ ]:
"""test = pd.read_csv('test_set.csv',dtype={'object_id':np.int32,'mjd':np.float16,\
                                                   'passband':np.int8,'flux':np.float32,'flux_err':np.float32,'detected':np.int8})"""

In [ ]:
"""number_chunk = 50000000
test[number_chunk*0:number_chunk*1].to_csv('test1.csv',index=False)
test[number_chunk*1:number_chunk*2].to_csv('test2.csv',index=False)
test[number_chunk*2:number_chunk*3].to_csv('test3.csv',index=False)
test[number_chunk*3:number_chunk*4].to_csv('test4.csv',index=False)
test[number_chunk*4:number_chunk*5].to_csv('test5.csv',index=False)
test[number_chunk*5:number_chunk*6].to_csv('test6.csv',index=False)
test[number_chunk*6:number_chunk*7].to_csv('test7.csv',index=False)
test[number_chunk*7:number_chunk*8].to_csv('test8.csv',index=False)
test[number_chunk*8:].to_csv('test9.csv',index=False)"""

# train

In [ ]:
gc.collect()

In [ ]:
train.head(5)

In [ ]:
agg_train = train.groupby(['object_id','passband'])[['flux_err','flux','detected']].agg(['count','min','max','std','sum','mean','median'])

In [ ]:
agg_train.head()

In [ ]:
agg_train.columns = [y+'_'+ x for x in ['flux_err','flux','detected'] for y in ['count','min','max','std','sum','mean','median']]

In [ ]:
agg_train = agg_train.reset_index(drop=False)

In [ ]:
agg_train.shape

# TRAIN

In [ ]:
%%time
train = sqldf("""SELECT A.*, B.mean_flux,B.std_flux FROM train A 
LEFT JOIN agg_train B
ON A.object_id = B.object_id AND A.passband = B.passband""")

In [ ]:
print(train.shape,agg_train.shape)

# region_minus_4

In [ ]:
%%time
# region_minus_4
temp = sqldf("""
SELECT D.object_id,D.passband,COUNT(*) as region_minus_4
FROM agg_train D 
LEFT JOIN (SELECT * FROM train B WHERE B.mean_flux - B.std_flux * 3 > B.flux) C
ON C.object_id = D.object_id AND C.passband = D.passband
GROUP BY C.object_id,C.passband
""")

In [ ]:
agg_train = sqldf("""SELECT A.*,B.region_minus_4 FROM agg_train A
LEFT JOIN temp B 
ON A.object_id = B.object_id AND A.passband = B.passband
""")

# region_minus_3

In [ ]:
%%time
#region_minus_3
temp = sqldf("""
SELECT D.object_id,D.passband,COUNT(*) as region_minus_3
FROM agg_train D 
LEFT JOIN (SELECT * FROM train B WHERE B.mean_flux - B.std_flux * 2 > B.flux AND B.mean_flux - B.std_flux * 3 < B.flux) C
ON C.object_id = D.object_id AND C.passband = D.passband
GROUP BY C.object_id,C.passband
""")

In [ ]:
agg_train = sqldf("""SELECT A.*,B.region_minus_3 FROM agg_train A
LEFT JOIN temp B 
ON A.object_id = B.object_id AND A.passband = B.passband
""")

# region_minus_2

In [ ]:
%%time
#region_minus_2
temp = sqldf("""
SELECT D.object_id,D.passband,COUNT(*) as region_minus_2
FROM agg_train D 
LEFT JOIN (SELECT * FROM train B WHERE B.mean_flux - B.std_flux * 1 > B.flux AND B.mean_flux - B.std_flux * 2 < B.flux) C
ON C.object_id = D.object_id AND C.passband = D.passband
GROUP BY C.object_id,C.passband
""")

In [ ]:
agg_train = sqldf("""SELECT A.*,B.region_minus_2 FROM agg_train A
LEFT JOIN temp B 
ON A.object_id = B.object_id AND A.passband = B.passband
""")

# region_minus_1

In [ ]:
%%time
#region_minus_1
temp = sqldf("""
SELECT D.object_id,D.passband,COUNT(*) as region_minus_1
FROM agg_train D 
LEFT JOIN (SELECT * FROM train B WHERE B.mean_flux - B.std_flux * 0 > B.flux AND B.mean_flux - B.std_flux * 1 < B.flux) C
ON C.object_id = D.object_id AND C.passband = D.passband
GROUP BY C.object_id,C.passband
""")

In [ ]:
agg_train = sqldf("""SELECT A.*,B.region_minus_1 FROM agg_train A
LEFT JOIN temp B 
ON A.object_id = B.object_id AND A.passband = B.passband
""")

# region_plus_1

In [ ]:
%%time
#region_plus_1
temp = sqldf("""
SELECT D.object_id,D.passband,COUNT(*) as region_plus_1
FROM agg_train D 
LEFT JOIN (SELECT * FROM train B WHERE B.mean_flux + B.std_flux * 0 < B.flux AND B.mean_flux + B.std_flux * 1 > B.flux) C
ON C.object_id = D.object_id AND C.passband = D.passband
GROUP BY C.object_id,C.passband
""")

In [ ]:
agg_train = sqldf("""SELECT A.*,B.region_plus_1 FROM agg_train A
LEFT JOIN temp B 
ON A.object_id = B.object_id AND A.passband = B.passband
""")

# region_plus_2

In [ ]:
%%time
#region_plus_2
temp = sqldf("""
SELECT D.object_id,D.passband,COUNT(*) as region_plus_2
FROM agg_train D 
LEFT JOIN (SELECT * FROM train B WHERE B.mean_flux + B.std_flux * 1 < B.flux AND B.mean_flux + B.std_flux * 2 > B.flux) C
ON C.object_id = D.object_id AND C.passband = D.passband
GROUP BY C.object_id,C.passband
""")

In [ ]:
agg_train = sqldf("""SELECT A.*,B.region_plus_2 FROM agg_train A
LEFT JOIN temp B 
ON A.object_id = B.object_id AND A.passband = B.passband
""")

# region_plus_3

In [ ]:
%%time
#region_plus_3
temp = sqldf("""
SELECT D.object_id,D.passband,COUNT(*) as region_plus_3
FROM agg_train D 
LEFT JOIN (SELECT * FROM train B WHERE B.mean_flux + B.std_flux * 2 < B.flux AND B.mean_flux + B.std_flux * 3 > B.flux) C
ON C.object_id = D.object_id AND C.passband = D.passband
GROUP BY C.object_id,C.passband
""")

In [ ]:
agg_train = sqldf("""SELECT A.*,B.region_plus_3 FROM agg_train A
LEFT JOIN temp B 
ON A.object_id = B.object_id AND A.passband = B.passband
""")

# region_plus_4

In [ ]:
%%time
#region_plus_4
temp = sqldf("""
SELECT D.object_id,D.passband,COUNT(*) as region_plus_4
FROM agg_train D 
LEFT JOIN (SELECT * FROM train B WHERE B.mean_flux + B.std_flux * 3 < B.flux ) C
ON C.object_id = D.object_id AND C.passband = D.passband
GROUP BY C.object_id,C.passband
""")

In [ ]:
agg_train = sqldf("""SELECT A.*,B.region_plus_4 FROM agg_train A
LEFT JOIN temp B 
ON A.object_id = B.object_id AND A.passband = B.passband
""")

In [ ]:
print(agg_train.shape,train.shape)

In [ ]:
agg_train.head()

In [ ]:
temp.shape

In [ ]:
del temp
gc.collect()

In [ ]:
del agg_train['min_detected'], agg_train['max_detected']